# PROBLEM 1: Topic Models
## Obtain Topic Models (K=10, 20, 50) for both datasets by running LDA and NMF methods; you can call libraries for both methods and dont have to use the ES index as source. For both LDA and NMF: print out for each topic the top 20 words (with probabilities)

The rest of of topic exercises and results are required only for the LDA topics:

- 20NG: how well the topics align with the 20NG label classes? This is not asking for a measurement, but rather for a visual inspection to determine what topics match well with what classes. 

Does this change if one increases the topics from 20 to 50?


### 1. Data Preprocessing using gensim

In [206]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-download-auto-examples-tutorials-run-lda-py

# imports

from sklearn.datasets import fetch_20newsgroups
from gensim import corpora, models, similarities, downloader

# Load the 20 newsgroups dataset
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
# truncate the dataset to 1000 documents for faster processing
# newsgroups.data = newsgroups.data[:1000]

print(type(newsgroups.data[0][:]))  # <class 'scipy.sparse.csr.csr_matrix'>
print(newsgroups.data[2][:])  # <class 'numpy.ndarray'>



<class 'str'>
well folks, my mac plus finally gave up the ghost this weekend after
starting life as a 512k way back in 1985.  sooo, i'm in the market for a
new machine a bit sooner than i intended to be...

i'm looking into picking up a powerbook 160 or maybe 180 and have a bunch
of questions that (hopefully) somebody can answer:

* does anybody know any dirt on when the next round of powerbook
introductions are expected?  i'd heard the 185c was supposed to make an
appearence "this summer" but haven't heard anymore on it - and since i
don't have access to macleak, i was wondering if anybody out there had
more info...

* has anybody heard rumors about price drops to the powerbook line like the
ones the duo's just went through recently?

* what's the impression of the display on the 180?  i could probably swing
a 180 if i got the 80Mb disk rather than the 120, but i don't really have
a feel for how much "better" the display is (yea, it looks great in the
store, but is that all "wow" or i

In [207]:
# gensim expects a list of lists of tokens, so we need to convert the sparse matrix to a list of lists
# %pip install nltk
# Tokenize the documents
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

for idx in range(len(newsgroups.data)):
    newsgroups.data[idx] = newsgroups.data[idx][:].lower()  # convert to lowercase
    newsgroups.data[idx] = tokenizer.tokenize(newsgroups.data[idx])  # split into words

print(newsgroups.data[2]) 
print(len(newsgroups.data[2])) # number of tokens in the document

['well', 'folks', 'my', 'mac', 'plus', 'finally', 'gave', 'up', 'the', 'ghost', 'this', 'weekend', 'after', 'starting', 'life', 'as', 'a', '512k', 'way', 'back', 'in', '1985', 'sooo', 'i', 'm', 'in', 'the', 'market', 'for', 'a', 'new', 'machine', 'a', 'bit', 'sooner', 'than', 'i', 'intended', 'to', 'be', 'i', 'm', 'looking', 'into', 'picking', 'up', 'a', 'powerbook', '160', 'or', 'maybe', '180', 'and', 'have', 'a', 'bunch', 'of', 'questions', 'that', 'hopefully', 'somebody', 'can', 'answer', 'does', 'anybody', 'know', 'any', 'dirt', 'on', 'when', 'the', 'next', 'round', 'of', 'powerbook', 'introductions', 'are', 'expected', 'i', 'd', 'heard', 'the', '185c', 'was', 'supposed', 'to', 'make', 'an', 'appearence', 'this', 'summer', 'but', 'haven', 't', 'heard', 'anymore', 'on', 'it', 'and', 'since', 'i', 'don', 't', 'have', 'access', 'to', 'macleak', 'i', 'was', 'wondering', 'if', 'anybody', 'out', 'there', 'had', 'more', 'info', 'has', 'anybody', 'heard', 'rumors', 'about', 'price', 'drops

In [208]:
# Remove numbers, but not words that contain numbers.
for ng in newsgroups.data:
    ng = [token for token in ng if not token.isdigit()]
    # Remove words that are only one character.
    ng = [token for token in ng if len(token) > 1]



print(newsgroups.data[2])  
print(len(newsgroups.data[2]))

['well', 'folks', 'my', 'mac', 'plus', 'finally', 'gave', 'up', 'the', 'ghost', 'this', 'weekend', 'after', 'starting', 'life', 'as', 'a', '512k', 'way', 'back', 'in', '1985', 'sooo', 'i', 'm', 'in', 'the', 'market', 'for', 'a', 'new', 'machine', 'a', 'bit', 'sooner', 'than', 'i', 'intended', 'to', 'be', 'i', 'm', 'looking', 'into', 'picking', 'up', 'a', 'powerbook', '160', 'or', 'maybe', '180', 'and', 'have', 'a', 'bunch', 'of', 'questions', 'that', 'hopefully', 'somebody', 'can', 'answer', 'does', 'anybody', 'know', 'any', 'dirt', 'on', 'when', 'the', 'next', 'round', 'of', 'powerbook', 'introductions', 'are', 'expected', 'i', 'd', 'heard', 'the', '185c', 'was', 'supposed', 'to', 'make', 'an', 'appearence', 'this', 'summer', 'but', 'haven', 't', 'heard', 'anymore', 'on', 'it', 'and', 'since', 'i', 'don', 't', 'have', 'access', 'to', 'macleak', 'i', 'was', 'wondering', 'if', 'anybody', 'out', 'there', 'had', 'more', 'info', 'has', 'anybody', 'heard', 'rumors', 'about', 'price', 'drops

In [209]:
%pip install nltk
import nltk
# nltk.download()

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\koola\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [210]:
# Lemmatize the documents
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
newsgroups.data = [[lemmatizer.lemmatize(token) for token in doc] for doc in newsgroups.data]

In [211]:
# Compute bigrams
from gensim.models import Phrases
bigrams = Phrases(newsgroups.data, min_count=5, threshold=100)
# bigrams = Phrases(newsgroups.data, min_count=20)
for idx in range(len(newsgroups.data)):
    for token in bigrams[newsgroups.data[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            newsgroups.data[idx].append(token)

In [212]:
# Removing words that appear in less than 20 docs OR in more than 50% of the docs.

# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(newsgroups.data)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
# dictionary.filter_extremes(no_below=20, no_above=0.5)

In [213]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in newsgroups.data]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 98832
Number of documents: 11314


### 2. Modeling

In [215]:
# We are ready to train the LDA model. We will first discuss how to set some of the training parameters.

# chunksize 
    # controls how many documents are processed at a time in the training algorithm. 
    # Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory. 
    # I’ve set chunksize = 2000, which is more than the amount of documents, so I process all the data in one go. 
    # Chunksize can however influence the quality of the model, as discussed in Hoffman and co-authors [2], but the difference was not substantial in this case.
# passes
    # controls how often we train the model on the entire corpus. 
    # Another word for passes might be “epochs”. iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document.
    # It is important to set the number of “passes” and “iterations” high enough.

from gensim.models import LdaModel

# Set training parameters.
num_topics = 20
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

# Train the model.
model = LdaModel(corpus=corpus, 
                 id2word=id2word, 
                 chunksize=chunksize, 
                 passes=passes, 
                 iterations=iterations, 
                 num_topics=num_topics, 
                 eval_every=eval_every)


In [ ]:
# Print the Keyword in the topics
topics = model.print_topics(num_words=4)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability

(0, '0.053*"the" + 0.037*"of" + 0.031*"to" + 0.027*"that"')
(1, '0.018*"france" + 0.012*"ryan" + 0.007*"bathroom" + 0.006*"bedroom"')
(2, '0.034*"please" + 0.033*"mail" + 0.027*"for" + 0.023*"thanks"')
(3, '0.521*"ax" + 0.038*"max" + 0.028*"q" + 0.022*"3"')
(4, '0.055*"m" + 0.042*"_" + 0.034*"w" + 0.023*"1"')
(5, '0.022*"cd" + 0.018*"adam" + 0.013*"ctrl" + 0.009*"da"')
(6, '0.040*"the" + 0.025*"and" + 0.021*"a" + 0.018*"to"')
(7, '0.003*"uoknor" + 0.003*"uoknor_edu" + 0.000*"dash" + 0.000*"seningen"')
(8, '0.089*"1" + 0.077*"0" + 0.057*"2" + 0.042*"3"')
(9, '0.004*"ethernet_card" + 0.004*"toshiba" + 0.002*"expansion_slot" + 0.002*"internal_modem"')
(10, '0.024*"space" + 0.014*"edu" + 0.008*"nasa" + 0.008*"at"')
(11, '0.054*"the" + 0.040*"a" + 0.035*"i" + 0.033*"to"')
(12, '0.035*"0" + 0.033*"game" + 0.032*"team" + 0.017*"player"')
(13, '0.026*"rider" + 0.015*"suck" + 0.011*"beach" + 0.006*"lciii"')
(14, '0.019*"char" + 0.009*"coast" + 0.007*"apr_1993" + 0.006*"mark"')
(15, '0.009*"comp

In [217]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -5.4757.
[([(0.05397378, 'the'),
   (0.039999828, 'a'),
   (0.035221323, 'i'),
   (0.032537278, 'to'),
   (0.021373285, 'it'),
   (0.018254673, 'is'),
   (0.017383188, 'and'),
   (0.017112624, 'that'),
   (0.01631468, 'you'),
   (0.015336333, 'of'),
   (0.012827275, 'in'),
   (0.011845945, 'have'),
   (0.0111008305, 'for'),
   (0.010212261, 'this'),
   (0.010098661, 't'),
   (0.009882227, 'be'),
   (0.00927743, 'if'),
   (0.008976932, 'on'),
   (0.008131061, 's'),
   (0.007927541, 'but')],
  -0.5332465596989693),
 ([(0.052510083, 'the'),
   (0.037010893, 'of'),
   (0.030909758, 'to'),
   (0.027391419, 'that'),
   (0.027091673, 'a'),
   (0.026754776, 'is'),
   (0.020334795, 'and'),
   (0.017405462, 'in'),
   (0.01518277, 'it'),
   (0.015003293, 'not'),
   (0.013101078, 'you'),
   (0.011706672, 'are'),
   (0.009435979, 'i'),
   (0.009406719, 'this'),
   (0.00922601, 'be'),
   (0.007360744, 'for'),
   (0.0069384393, 'have'),
   (0.006410113, 'we'),
   (0.006116928